In [1]:
import pandas as pd
import numpy as np
from catnip.fla_redshift import FLA_Redshift
from sqlalchemy import null
from datetime import datetime

from prefect.blocks.system import Secret
from typing import Dict
from concurrent.futures import ThreadPoolExecutor

In [2]:
def get_redshift_credentials() -> Dict:

    cred_dict = {
        "dbname": Secret.load("stellar-redshift-db-name").get(),
        "host": Secret.load("stellar-redshift-host").get(),
        "port": 5439,
        "user": Secret.load("stellar-redshift-user-name").get(),
        "password": Secret.load("stellar-redshift-password").get(),

        "aws_access_key_id": Secret.load("fla-s3-aws-access-key-id-east-1").get(),
        "aws_secret_access_key": Secret.load("fla-s3-aws-secret-access-key-east-1").get(),
        "bucket": Secret.load("fla-s3-bucket-name-east-1").get(),
        "subdirectory": "us-east-1",

        "verbose": False,
    }

    return cred_dict

with ThreadPoolExecutor(1) as pool:
    rs_creds = pool.submit(lambda: get_redshift_credentials()).result()

In [3]:
df = pd.read_csv("C:\\Users\\riffere\\OneDrive - Florida Panthers\\Documents\\LTP\\Kids_Club.csv")
FLA_Redshift(**rs_creds).write_to_warehouse(df = df, table_name= "community_kidsclub_data")

In [4]:
q = """
WITH initial AS (
    SELECT
        LOWER(parent_email) AS parent_email,
        CASE
            WHEN LISTAGG(kc_group) LIKE '%All-Star%' THEN 'All-Star'
            WHEN LISTAGG(kc_group) LIKE '%Pro%' THEN 'Pro'
            ELSE 'Rookie'
        END AS kc_group_final
    FROM
        custom.community_kidsclub_data
    GROUP BY
        LOWER(parent_email)
),
first_kids_club_season AS (
    SELECT
        initial.parent_email,
        kc_group_final AS kc_group,
        MIN(transaction_date) AS transaction_date,
        MIN(season) AS season
    FROM
        initial
    LEFT JOIN
        custom.community_kidsclub_data
    ON
        LOWER(initial.parent_email) = LOWER(community_kidsclub_data.parent_email)
    GROUP BY
        initial.parent_email, 
        kc_group_final
),
acct_rev_info AS (
    SELECT
        contacts.email,
        createdon AS add_date,
        CASE
            WHEN ticket.season LIKE '2021-22' THEN gross_revenue
            ELSE 0
        END AS "gross_rev_2122",
        CASE
            WHEN ticket.season LIKE '2022-23' THEN gross_revenue
            ELSE 0
        END AS "gross_rev_2223",
        CASE
            WHEN ticket.season LIKE '2023-24' THEN gross_revenue
            ELSE 0
        END AS "gross_rev_2324",
        CASE
            WHEN ticket.season LIKE '2024-25' THEN gross_revenue
            ELSE 0
        END AS "gross_rev_2425",
        CASE
            WHEN ticket.season LIKE '2021-22' AND "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2122",
        CASE
            WHEN ticket.season LIKE '2022-23' AND "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2223",
        CASE
            WHEN ticket.season LIKE '2023-24' AND "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2324",
        CASE
            WHEN ticket.season LIKE '2024-25' AND "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2425"
    FROM
        custom.cth_v_historical_ticket ticket
    INNER JOIN
        custom.korepss_externalsystemtocontact ext ON ticket.purchaser_ticketing_id = ext.externalcontactid
    INNER JOIN
        custom.korepss_contacts contacts ON ext.crmcontactid = contacts.contactid
),
acct_rev_agg AS (
    SELECT
        email,
        MIN(add_date) AS add_date,
        SUM(gross_rev_2122) AS gross_rev_2122,
        SUM(gross_rev_2223) AS gross_rev_2223,
        SUM(gross_rev_2324) AS gross_rev_2324,
        SUM(gross_rev_2425) AS gross_rev_2425,
        MAX(is_stm_2122) AS is_stm_2122,
        MAX(is_stm_2223) AS is_stm_2223,
        MAX(is_stm_2324) AS is_stm_2324,
        MAX(is_stm_2425) AS is_stm_2425
    FROM
        acct_rev_info
    GROUP BY
        email
)
SELECT
    parent_email,
    kc_group,
    transaction_date::date,
    first_kids_club_season.season,
    add_date,
    COALESCE(gross_rev_2122, 0) AS gross_rev_2122,
    COALESCE(gross_rev_2223, 0) AS gross_rev_2223,
    COALESCE(gross_rev_2324, 0) AS gross_rev_2324,
    COALESCE(gross_rev_2425, 0) AS gross_rev_2425,
    COALESCE(is_stm_2122, 0) AS is_stm_2122,
    COALESCE(is_stm_2223, 0) AS is_stm_2223,
    COALESCE(is_stm_2324, 0) AS is_stm_2324,
    COALESCE(is_stm_2425, 0) AS is_stm_2425
FROM
    first_kids_club_season
LEFT JOIN
    acct_rev_agg ON first_kids_club_season.parent_email = acct_rev_agg.email
WHERE
    first_kids_club_season.parent_email IS NOT NULL AND first_kids_club_season.parent_email <> ''
"""

In [5]:
df = FLA_Redshift(**rs_creds).query_warehouse(sql_string = q)

In [7]:
df['add_date'].fillna(datetime.strptime('2030-01-01', "%Y-%m-%d"), inplace = True)
df['add_date'] = pd.to_datetime(df['add_date']).dt.date
df['transaction_date'] = pd.to_datetime(df['transaction_date']).dt.date

C:\Users\riffere\AppData\Local\Temp\ipykernel_3724\4015671550.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['add_date'].fillna(datetime.strptime('2030-01-01', "%Y-%m-%d"), inplace = True)


In [8]:
df['pre_kids_club_email'] = df.apply(lambda row: 0 if row['transaction_date'] == '' else (1 if row['add_date'] < row['transaction_date'] else 0), axis = 1)
df['post_kids_club_email'] = df.apply(lambda row: 0 if row['transaction_date'] == '' else (1 if row['add_date'] >= row['transaction_date'] else 0), axis = 1)

In [10]:
season_dict = {
    "2021-22" : 1,
    "2022-23" : 2,
    "2023-24" : 3,
    "2024-25" : 4
}

# sum season tickets before/after
for key, value in season_dict.items():
    col_list = list(df.columns[5:(5+value)])
    # print(col_list)
    df.loc[(df['season'] == key), 'pre_kids_club_revenue'] =  df[col_list].sum(axis = 1)

for key, value in season_dict.items():
    col_list = list(df.columns[(5 + value):9])
    # print(col_list)
    df.loc[(df['season'] == key), 'post_kids_club_revenue'] =  df[col_list].sum(axis = 1)

# sum revenue before/after
for key, value in season_dict.items():
    col_list = list(df.columns[9:(9+value)])
    # print(col_list)
    df.loc[(df['season'] == key), 'pre_kids_club_tickets'] =  df[col_list].sum(axis = 1)


for key, value in season_dict.items():
    col_list = list(df.columns[(9 + value):13])
    # print(col_list)
    df.loc[(df['season'] == key), 'post_kids_club_tickets'] =  df[col_list].sum(axis = 1)

In [12]:
FLA_Redshift(**rs_creds).write_to_warehouse(df = df, table_name= "community_v_kidsclub")